In [1]:
#!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

In [2]:
import os
os.environ['DGLBACKEND'] = 'pytorch'

In [3]:
import torch
from torch import nn

import dgl
import dgl.function as fn

import networkx as nx
from matplotlib import pyplot as plt
import numpy as np

import os
import gc

# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
def printNPZ(npz):
    for kw in npz.files:
        print(kw, npz[kw])

In [5]:
dirName = './HiraiwaModel_chem20220916_150816/'
savedirName = dirName + 'ActiveNet_vp_rotsym_dropout/'
os.makedirs(savedirName, exist_ok=True)

params = np.load(dirName+'params.npz')
#traj = np.load(dirName+'result.npz')

printNPZ(params)
#printNPZ(traj)

v0 1.0
r 1.0
D 0.1
A 0.0
L 20
rho 1.0
beta 1.0
A_CFs [0.1 0.9]
A_CIL 0.0
cellType_ratio [0.5 0.5]
quiv_colors ['k' 'r']
kappa 0.5
A_Macdonalds [2.  0.2]
batch_size 400
state_size 3
brownian_size 1
periodic True
t_max 1000
methodSDE heun
isIto False
stepSDE 0.01


In [6]:
if params['periodic']:
    L = torch.tensor(params['L'])
    def calc_dr(r1, r2):
        dr = torch.remainder((r1 - r2), L)
        dr[dr > L/2] = dr[dr > L/2] - L
        return dr
else:
    def calc_dr(r1, r2):
        return r1 - r2
    
def makeGraph(x_data, r_thresh):
        Ndata = x_data.size(0)
        dx = calc_dr(torch.unsqueeze(x_data, 0), torch.unsqueeze(x_data, 1))
        dx = torch.sum(dx**2, dim=2)
        edges = torch.argwhere(dx < r_thresh/2)
        return dgl.graph((edges[:,0], edges[:,1]), num_nodes=Ndata)

In [7]:
subdir_list = [f.path for f in os.scandir(dirName) if f.is_dir()]

print(subdir_list)

datadir_list = [f for f in subdir_list if 'result.npz' in [ff.name for ff in os.scandir(f)]]

print(datadir_list)

['./HiraiwaModel_chem20220916_150816/20221013_130509', './HiraiwaModel_chem20220916_150816/20221013_222508', './HiraiwaModel_chem20220916_150816/20221014_020927', './HiraiwaModel_chem20220916_150816/20221014_055334', './HiraiwaModel_chem20220916_150816/20221014_093729', './HiraiwaModel_chem20220916_150816/ActiveNet_celltypes', './HiraiwaModel_chem20220916_150816/ActiveNet_vp_rotsym', './HiraiwaModel_chem20220916_150816/20221022_000642', './HiraiwaModel_chem20220916_150816/ActiveNet_vp_rotsym_dropout']
['./HiraiwaModel_chem20220916_150816/20221013_130509', './HiraiwaModel_chem20220916_150816/20221013_222508', './HiraiwaModel_chem20220916_150816/20221014_020927', './HiraiwaModel_chem20220916_150816/20221014_055334', './HiraiwaModel_chem20220916_150816/20221014_093729', './HiraiwaModel_chem20220916_150816/20221022_000642']


In [8]:
dr_thresh = 4
dt = 1
batch_size = 8

N_data = len(datadir_list)

#TR_VA_rate = np.array([0.6, 0.2])

TR_last = 3
VA_last = 4

shuffle_inds = np.arange(N_data, dtype=int)
np.random.shuffle(shuffle_inds)

train_inds = shuffle_inds[:TR_last]
valid_inds = shuffle_inds[TR_last:VA_last]
test_inds = shuffle_inds[VA_last:]

celltype_lst = []

train_x = []
valid_x = []
test_x = []

train_y = []
valid_y = []
test_y = []

train_i_dir = []
valid_i_dir = []
test_i_dir = []

for i_dir, subdirName in enumerate(datadir_list):
    
    traj = np.load(subdirName+'/result.npz')
    
    celltype_lst.append(torch.tensor(traj['celltype_label']).view(-1,1))

    xy_t = torch.tensor(traj['xy'][:-1,:,:])
    v_t = calc_dr(torch.tensor(traj['xy'][1:,:,:]), torch.tensor(traj['xy'][:-1,:,:])) / dt
    p_t = torch.unsqueeze(torch.tensor(traj['theta'][:-1,:]), dim=2)
    w_t = torch.unsqueeze(torch.tensor((traj['theta'][1:,:]-traj['theta'][:-1,:])%(2*np.pi)/dt), dim=2)
    
    if i_dir in train_inds:
        train_x.append(torch.concat((xy_t, p_t), -1))
        train_y.append(torch.concat((v_t, w_t), -1))
        train_i_dir.append(torch.ones([xy_t.size(0)])*i_dir)

    if i_dir in valid_inds:
        valid_x.append(torch.concat((xy_t, p_t), -1))
        valid_y.append(torch.concat((v_t, w_t), -1))
        valid_i_dir.append(torch.ones([xy_t.size(0)])*i_dir)
        
    if i_dir in test_inds:
        test_x.append(torch.concat((xy_t, p_t), -1))
        test_y.append(torch.concat((v_t, w_t), -1))
        test_i_dir.append(torch.ones([xy_t.size(0)])*i_dir)
    
train_dataset = torch.utils.data.TensorDataset(
    torch.concat(train_x, 0), 
    torch.concat(train_y, 0), 
    torch.concat(train_i_dir, 0))

valid_dataset = torch.utils.data.TensorDataset(
    torch.concat(valid_x, 0), 
    torch.concat(valid_y, 0), 
    torch.concat(valid_i_dir, 0))

test_dataset = torch.utils.data.TensorDataset(
    torch.concat(test_x, 0), 
    torch.concat(test_y, 0), 
    torch.concat(test_i_dir, 0))

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, pin_memory=True)
valid_data = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

del train_x, train_y, train_i_dir, train_dataset
del valid_x, valid_y, valid_i_dir, valid_dataset
del test_x, test_y, test_i_dir, test_dataset
gc.collect()

#print(data)
#print(data.num_graphs)
#print(data.x)
#print(data.y)
#print(data.edge_index)

165

In [9]:
print(train_inds)

[2 5 4]


In [10]:
dir(train_data)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_get_shared_seed',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_i

In [11]:
def plotGraph(data):

    # networkxのグラフに変換
    nxg = dgl.to_networkx(data)

    # 可視化のためのページランク計算
    pr = nx.pagerank(nxg)
    pr_max = np.array(list(pr.values())).max()

    # 可視化する際のノード位置
    draw_pos = nx.spring_layout(nxg, seed=0) 

    # ノードの色設定
    cmap = plt.get_cmap('tab10')
    labels = data.y.numpy()
    colors = [cmap(l) for l in labels]

    # 図のサイズ
    fig0 = plt.figure(figsize=(10, 10))

    # 描画
    nx.draw_networkx_nodes(nxg, 
                          draw_pos,
                          node_size=[v / pr_max * 1000 for v in pr.values()])#,
                          #node_color=colors, alpha=0.5)
    nx.draw_networkx_edges(nxg, draw_pos, arrowstyle='-', alpha=0.2)
    nx.draw_networkx_labels(nxg, draw_pos, font_size=10)

    #plt.title('KarateClub')
    plt.show()

    return fig0

In [12]:
class NeuralNet(nn.Module):
    def __init__(self, in_channels, out_channels, Nchannels, dropout=0, flgBias=False):
        super(NeuralNet, self).__init__()

        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0
        
        self.layer1 = nn.Linear(in_channels, Nchannels, bias=flgBias)
        self.layer2 = nn.Linear(Nchannels, Nchannels, bias=flgBias)
        self.layer3 = nn.Linear(Nchannels, Nchannels, bias=flgBias)
        self.layer4 = nn.Linear(Nchannels, out_channels, bias=flgBias)

        self.activation = nn.ReLU()

    def reset_parameters(self):
        self.layer1.reset_parameters()
        self.layer2.reset_parameters()
        self.layer3.reset_parameters()
        self.layer4.reset_parameters()
        #nn.init.zeros_(self.layer1.weight)
        #nn.init.zeros_(self.layer2.weight)
        #nn.init.zeros_(self.layer3.weight)
        #nn.init.zeros_(self.layer4.weight)
        
    def forward(self, x):
        out = self.activation(self.layer1(x))
        if self.dropout:
            out = self.dropout(out)
        out = self.activation(self.layer2(out))
        if self.dropout:
            out = self.dropout(out)
        out = self.activation(self.layer3(out))
        if self.dropout:
            out = self.dropout(out)
        out = self.layer4(out)

        return out

class ActiveNet(nn.Module):
    def __init__(self, xy_dim, r, dropout=0, Nchannels=128):
        super().__init__()

        self.interactNN = NeuralNet(xy_dim*2 + 2, xy_dim, Nchannels, dropout, False)

        self.thetaDotNN = NeuralNet(xy_dim*2 + 2, 1, Nchannels, dropout, False)
        
        self.selfpropel = nn.Parameter(torch.tensor(0.0, requires_grad=True, device=device))

        #self.Normalizer = nn.Softmax(dim=1)

        self.xy_dim = xy_dim
        
        self.r = r

        self.reset_parameters()

    def reset_parameters(self):
        self.interactNN.reset_parameters()

        self.thetaDotNN.reset_parameters()
        
        nn.init.uniform_(self.selfpropel)

        #self.bias.data.zero_()
        
    def load_celltypes(self, celltype):
        self.celltype = celltype

    def calc_message(self, edges):
        dx = calc_dr(edges.dst['x'], edges.src['x'])

        costheta = torch.cos(edges.dst['theta'])
        sintheta = torch.sin(edges.dst['theta'])

        dx_para = costheta * dx[..., :1] + sintheta * dx[..., 1:]
        dx_perp = costheta * dx[..., 1:] - sintheta * dx[..., :1]

        p_para_src = torch.cos(edges.src['theta'] - edges.dst['theta'])
        p_perp_src = torch.sin(edges.src['theta'] - edges.dst['theta'])

        rot_m_v = self.interactNN(torch.concat((dx_para, dx_perp, 
                                                p_para_src, p_perp_src,
                                                edges.dst['type'], edges.src['type']), -1))

        m_v = torch.concat((costheta * rot_m_v[..., :1] - sintheta * rot_m_v[..., 1:],
                            costheta * rot_m_v[..., 1:] + sintheta * rot_m_v[..., :1]), -1)

        m_theta = self.thetaDotNN(torch.concat((dx_para, dx_perp, 
                                                p_para_src, p_perp_src, 
                                                edges.dst['type'], edges.src['type']), -1))
        
        return {'m': torch.concat((m_v, m_theta), -1)}
        
    def forward(self, xv):
        r_g = makeGraph(xv[..., :self.xy_dim], self.r/2)
        r_g.ndata['x'] = xv[..., :self.xy_dim]
        r_g.ndata['theta'] = xv[..., self.xy_dim:(self.xy_dim+1)]
        r_g.ndata['type'] = self.celltype
        r_g.update_all(self.calc_message, fn.sum('m', 'a'))
        r_g.ndata['a'][..., :self.xy_dim] = r_g.ndata['a'][..., :self.xy_dim] + self.selfpropel * torch.concat((torch.cos(r_g.ndata['theta']), torch.sin(r_g.ndata['theta'])), -1)
        
        return r_g.ndata['a']



In [13]:
def myLoss(out, target):
    dv = torch.sum(torch.square(out[..., :xy_dim] - target[..., :xy_dim]), dim=-1)
    dcos = 1 - torch.cos(out[..., xy_dim] - target[..., xy_dim])
    return torch.mean(dv + dcos)

In [14]:
# モデルのインスタンス生成
xy_dim = 2

model = ActiveNet(xy_dim, dr_thresh, dropout=0.2, Nchannels=128).to(device)
# input data
#data = dataset[0]

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

val_loss_log = []

val_loss_min = np.Inf

# learnig loop
for epoch in range(300):
    model.train()
    for batch_x, batch_y, batch_i_dir in train_data:
        optimizer.zero_grad()
        loss = 0
        for ib in range(batch_x.size(0)):
            model.load_celltypes(celltype_lst[int(batch_i_dir[ib])].to(device))
            out = model(batch_x[ib].to(device))
            loss = loss + myLoss(out, batch_y[ib].to(device))
        loss = loss / batch_x.size(0)
        loss.backward()
        optimizer.step()
    model.eval()
    val_loss = 0
    val_count = 0
    with torch.no_grad():
        for batch_x, batch_y, batch_i_dir in valid_data:
            for ib in range(batch_x.size(0)):
                model.load_celltypes(celltype_lst[int(batch_i_dir[ib])].to(device))
                val_out = model(batch_x[ib].to(device))
                val_loss = val_loss + myLoss(val_out, batch_y[ib].to(device))
            val_count = val_count + batch_x.size(0)
    val_loss = val_loss/val_count
    print('Epoch %d | train Loss: %.4f | valid Loss: %.4f' % (epoch, loss.item(), val_loss.item()))
    val_loss_log.append(val_loss.cpu().item())
    if val_loss.item() < val_loss_min:
        stored_model = model
        val_loss_min = val_loss.item()

Epoch 0 | train Loss: 0.1749 | valid Loss: 0.1986
Epoch 1 | train Loss: 0.1627 | valid Loss: 0.1898
Epoch 2 | train Loss: 0.1666 | valid Loss: 0.1861
Epoch 3 | train Loss: 0.1626 | valid Loss: 0.1818
Epoch 4 | train Loss: 0.1640 | valid Loss: 0.1815
Epoch 5 | train Loss: 0.1602 | valid Loss: 0.1802
Epoch 6 | train Loss: 0.1576 | valid Loss: 0.1785
Epoch 7 | train Loss: 0.1591 | valid Loss: 0.1786
Epoch 8 | train Loss: 0.1554 | valid Loss: 0.1770
Epoch 9 | train Loss: 0.1544 | valid Loss: 0.1773
Epoch 10 | train Loss: 0.1535 | valid Loss: 0.1764
Epoch 11 | train Loss: 0.1549 | valid Loss: 0.1768
Epoch 12 | train Loss: 0.1527 | valid Loss: 0.1757
Epoch 13 | train Loss: 0.1529 | valid Loss: 0.1767
Epoch 14 | train Loss: 0.1497 | valid Loss: 0.1765
Epoch 15 | train Loss: 0.1515 | valid Loss: 0.1758
Epoch 16 | train Loss: 0.1500 | valid Loss: 0.1756
Epoch 17 | train Loss: 0.1490 | valid Loss: 0.1756
Epoch 18 | train Loss: 0.1483 | valid Loss: 0.1746
Epoch 19 | train Loss: 0.1461 | valid Los

Epoch 160 | train Loss: 0.1301 | valid Loss: 0.1696
Epoch 161 | train Loss: 0.1302 | valid Loss: 0.1699
Epoch 162 | train Loss: 0.1301 | valid Loss: 0.1693
Epoch 163 | train Loss: 0.1292 | valid Loss: 0.1695
Epoch 164 | train Loss: 0.1310 | valid Loss: 0.1692
Epoch 165 | train Loss: 0.1285 | valid Loss: 0.1691
Epoch 166 | train Loss: 0.1329 | valid Loss: 0.1700
Epoch 167 | train Loss: 0.1301 | valid Loss: 0.1696
Epoch 168 | train Loss: 0.1299 | valid Loss: 0.1696
Epoch 169 | train Loss: 0.1296 | valid Loss: 0.1697
Epoch 170 | train Loss: 0.1310 | valid Loss: 0.1693
Epoch 171 | train Loss: 0.1292 | valid Loss: 0.1693
Epoch 172 | train Loss: 0.1295 | valid Loss: 0.1695
Epoch 173 | train Loss: 0.1297 | valid Loss: 0.1690
Epoch 174 | train Loss: 0.1298 | valid Loss: 0.1690
Epoch 175 | train Loss: 0.1287 | valid Loss: 0.1690
Epoch 176 | train Loss: 0.1289 | valid Loss: 0.1694
Epoch 177 | train Loss: 0.1288 | valid Loss: 0.1692
Epoch 178 | train Loss: 0.1283 | valid Loss: 0.1689
Epoch 179 | 

In [15]:
model.state_dict()

OrderedDict([('selfpropel', tensor(0.3957, device='cuda:0')),
             ('interactNN.layer1.weight',
              tensor([[ 7.2800e-01, -4.4048e-01, -3.6843e-01, -1.2402e-01, -9.2603e-02,
                       -2.1627e-01],
                      [-1.8960e-01,  2.7599e-01,  1.8265e-01,  9.7410e-02, -3.8153e-01,
                       -3.9291e-02],
                      [-4.2725e-01,  3.7821e-01, -3.9270e-01, -6.7720e-02, -4.8003e-02,
                       -6.4968e-01],
                      [ 2.5824e-01, -3.1067e-01,  2.2684e-01, -1.4650e-02, -6.3404e-03,
                       -3.0250e-01],
                      [ 1.6821e-01, -8.0436e-02, -1.1615e-01, -3.9521e-01, -1.4984e-01,
                       -8.2637e-03],
                      [-2.7781e-01, -3.3042e-01, -1.0342e-01,  4.7966e-02, -1.9153e-01,
                       -3.4227e-01],
                      [-1.5052e-01, -2.2708e-01, -1.9637e-01,  3.0579e-01, -6.9431e-02,
                       -3.7566e-02],
                     

In [16]:
# モデルを評価モードに設定
stored_model.eval()

# 推論
test_loss = 0
test_count = 0
with torch.no_grad():
    for batch_x, batch_y, batch_i_dir in test_data:
        for ib in range(batch_x.size(0)):
            model.load_celltypes(celltype_lst[int(batch_i_dir[ib])].to(device))
            test_out = model(batch_x[ib].to(device))
            test_loss = test_loss + myLoss(test_out, batch_y[ib].to(device))
        test_count = test_count + batch_x.size(0)
test_loss = test_loss/test_count
print('test Loss: %.4f' % (test_loss.item()))

test Loss: 0.1667


In [17]:
import pickle
import datetime

now = datetime.datetime.now()
nowstr = now.strftime('%Y%m%d_%H%M%S')

os.makedirs(savedirName + nowstr + '/', exist_ok=)

SyntaxError: invalid syntax (2675390831.py, line 7)

In [ ]:
stored_model.selfpropel.detach()

In [ ]:
stored_model = stored_model.to('cpu')

filename1 = savedirName + nowstr + '/' + nowstr + '_Model.pkl'
with open(filename1, "wb") as f:
    pickle.dump(stored_model, f)

filename1_2 = savedirName + nowstr + '/' + nowstr + '_Model.pt'
torch.save(stored_model, filename1_2)

filename2 = savedirName + nowstr + '/' + nowstr
torch.save(stored_model.interactNN.state_dict(), filename2 + '_interactNN.pkl')
torch.save(stored_model.thetaDotNN.state_dict(), filename2 + '_thetaDotNN.pkl')
torch.save(stored_model.selfpropel.detach(), filename2 + '_selfpropel.pkl')

filename3 = savedirName + nowstr + '/' + nowstr + '_Separation.npz'
np.savez(filename3, dr_thresh=dr_thresh, batch_size=batch_size,
         train_inds=train_inds, valid_inds=valid_inds, test_inds=test_inds, 
         val_loss_log=val_loss_log, test_loss=test_loss.cpu())

filename4 = savedirName + nowstr + '/' + nowstr + '_optimizer.pt'
torch.save(optimizer, filename4)